In [ ]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar


--2020-07-27 13:24:00--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.27MB/s    in 0.7s    

2020-07-27 13:24:01 (1.27 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Outdoors_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE|2015-01-31 00:08:00|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...| 

# Count the number of records (rows) in the dataset.

In [ ]:
df.count()

2302401

In [ ]:
# Drop null values
df = df.dropna()
# df = df.dropDuplicates()
# df.show()
print(df.count())

2302173


# Transform the dataset to fit the tables in the schema file

In [ ]:
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# Changing datatypes
# Source Link: https://stackoverflow.com/questions/32284620/how-to-change-a-dataframe-column-from-string-type-to-double-type-in-pyspark

from pyspark.sql.types import TimestampType, IntegerType

df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", df["total_votes"].cast(IntegerType()))\
      
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



### Review_id Table

# Level 2

### All Customers

In [ ]:
#  Viewing ALL CUSTOMERS on review date, customer, id, verified purchase, product title, star rating, helpful rating, total votes, and in vine program

analysis_df = df.select(["review_date","customer_id", "verified_purchase", "product_title","star_rating","helpful_votes","total_votes", "vine"])
analysis_df.show()

+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2015-01-31 00:08:00|   18446823|                Y|Stearns Youth Boa...|          4|            0|          0|   N|
|2015-01-31 00:08:00|   13724367|                Y|Primal Wear Men's...|          5|            0|          0|   N|
|2015-01-31 00:08:00|   51001958|                Y|Osprey Hydraulics...|          4|            0|          0|   N|
|2015-01-31 00:08:00|   32866903|                Y|CamelBak eddy .75...|          3|            1|          1|   N|
|2015-01-31 00:08:00|   30907790|                Y|Children Black Re...|          1|            0|          0|   N|
|2015-01-31 00:08:00|   20232229|                Y|Ibera Bicycle Tri...|

In [ ]:
# Summary statistics for ALL CUSTOMERS for star_rating, helpful votes, total votes
summary_analysis_df = analysis_df.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for ALL CUSTOMERS")
summary_analysis_df.show()

Summary statistics for ALL CUSTOMERS
+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|           2302173|           2302173|           2302173|
|   mean| 4.240025402087506| 1.846969363292854| 2.294820154697323|
| stddev|1.2101688438734601|12.770683948304539|13.628679978937035|
|    min|                 1|                 0|                 0|
|    max|                 5|              2703|              2751|
+-------+------------------+------------------+------------------+



In [ ]:
# Ordering Star rating wth 5 being the highest for ALL CUSTOMERS

from pyspark.sql.functions import desc

star_rating_df = analysis_df.orderBy(analysis_df.star_rating.desc())
star_rating_df.show()


+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2015-01-31 00:08:00|   36542249|                Y|RAD Cycle Product...|          5|            1|          1|   N|
|2015-01-31 00:08:00|   33495535|                Y|20L/33L- Most Dur...|          5|            0|          0|   N|
|2015-01-31 00:08:00|   16687947|                Y|XLC Quick Release...|          5|            0|          0|   N|
|2015-01-31 00:08:00|   13724367|                Y|Primal Wear Men's...|          5|            0|          0|   N|
|2015-01-31 00:08:00|   11373767|                Y|Timbuk2 Aviator T...|          5|            0|          1|   N|
|2015-01-31 00:08:00|   32421041|                Y|K2 Skate Men's F....|

### Vine Customers

In [ ]:
# Viewing VINE CUSTOMERS

# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_analysis_df = analysis_df.filter(col("vine")  == "Y")
vine_analysis_df.show()



+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2015-01-31 00:08:00|   37388532|                N|The Alter Ego Per...|          5|            2|          5|   Y|
|2015-01-31 00:08:00|   44451381|                N|Slumberjack Bound...|          5|            0|          0|   Y|
|2015-01-31 00:08:00|   17304105|                N|Thule EnRoute Blu...|          5|            3|          4|   Y|
|2015-01-31 00:08:00|   51016139|                N|Slumberjack Bound...|          4|            0|          0|   Y|
|2015-01-31 00:08:00|   31691563|                N|Timberjill 20 Deg...|          5|            0|          0|   Y|
|2015-01-31 00:08:00|   50046574|                N|The Alter Ego Per...|

In [ ]:
# Summary statistics for VINE CUSTOMERS for star_rating, helpful votes, total votes

vine_summary_analysis_df = vine_analysis_df.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for VINE CUSTOMERS")
vine_summary_analysis_df.show()

Summary statistics for VINE CUSTOMERS
+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|              3137|              3137|              3137|
|   mean| 4.372967803634046| 3.058335989799171|3.9190309212623524|
| stddev|0.8186341579032859|12.566739239470186|13.562137725035525|
|    min|                 1|                 0|                 0|
|    max|                 5|               326|               340|
+-------+------------------+------------------+------------------+



In [ ]:
# Top 20 rated products with the Top helpful votes for VINE CUSTOMERS

from pyspark.sql.functions import desc

vine_helpful_votes_df = vine_analysis_df.orderBy(vine_analysis_df.helpful_votes.desc())
vine_helpful_votes_df = vine_helpful_votes_df.filter('star_rating = 5')

print("Top 20 rated products with the Top helpful votes for VINE CUSTOMERS")
vine_helpful_votes_df.show()


Top 20 rated products with the Top helpful votes for VINE CUSTOMERS
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2014-01-08 00:08:00|   51999404|                N|Thule Urban Glide...|          5|          326|        340|   Y|
|2012-01-01 00:08:00|   49805190|                N|Zippo Flex Neck U...|          5|          170|        181|   Y|
|2010-01-19 00:09:00|   51859664|                N|  YBIKE Balance Bike|          5|          140|        153|   Y|
|2015-01-08 00:05:00|   13722260|                N|Thule Vertex XT H...|          5|          123|        128|   Y|
|2014-01-21 00:06:00|   51037469|                N|Mountain House Ju...|          5|          118|        126|   Y|
|201

In [ ]:
#  Converting into Pandas DataFrame to get the complete product title name 
vine_helpful_votes_df.select("product_title").toPandas().head(3)
# Confirmed full name on Amazon

,product_title
0,Thule Urban Glide - Jogging Stroller
1,Zippo Flex Neck Utility Lighter
2,YBIKE Balance Bike


In [ ]:
# Worst 20 rated products with the Top helpful votes for VINE CUSTOMERS

from pyspark.sql.functions import desc

low_vine_helpful_votes_df = vine_analysis_df.orderBy(vine_analysis_df.helpful_votes.desc())
low_vine_helpful_votes_df = low_vine_helpful_votes_df.filter('star_rating = 1')

print("Worst 20 rated products with the Top helpful votes for VINE CUSTOMERS")
low_vine_helpful_votes_df.show()


Worst 20 rated products with the Top helpful votes for VINE CUSTOMERS
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2013-01-31 00:05:00|   53090839|                N|Runtastic Bike Ca...|          1|           18|         18|   Y|
|2013-01-04 00:12:00|   51036953|                N|Allen Sports Prem...|          1|           17|         20|   Y|
|2014-01-08 00:01:00|   38056064|                N|Wahoo Balance Blu...|          1|           11|         12|   Y|
|2013-01-02 00:12:00|   52761853|                N|KOR Nava BPA Free...|          1|            7|         15|   Y|
|2012-01-27 00:07:00|   51002192|                N|Energizer Weather...|          1|            6|          9|   Y|
|2

In [ ]:
#  Converting into Pandas DataFrame to get the complete product title name 
low_vine_helpful_votes_df.select("product_title").toPandas().head(3)
# Confirmed full name on Amazon

,product_title
0,"Runtastic Bike Case for iPhone 4, 4S, 5 (Black)"
1,Allen Sports Premier 4-Bike Trunk Rack
2,Wahoo Balance Bluetooth Smart Scale for iPhone...


### Non-Vine customers

In [ ]:
# Viewing NON-VINE CUSTOMERS
non_vine_analysis_df = analysis_df.filter(col("vine")  == "N")
non_vine_analysis_df.show()



+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2015-01-31 00:08:00|   18446823|                Y|Stearns Youth Boa...|          4|            0|          0|   N|
|2015-01-31 00:08:00|   13724367|                Y|Primal Wear Men's...|          5|            0|          0|   N|
|2015-01-31 00:08:00|   51001958|                Y|Osprey Hydraulics...|          4|            0|          0|   N|
|2015-01-31 00:08:00|   32866903|                Y|CamelBak eddy .75...|          3|            1|          1|   N|
|2015-01-31 00:08:00|   30907790|                Y|Children Black Re...|          1|            0|          0|   N|
|2015-01-31 00:08:00|   20232229|                Y|Ibera Bicycle Tri...|

In [ ]:
# Summary statistics for NON-VINE CUSTOMERS for star_rating, helpful votes, total votes

non_vine_summary_analysis_df = non_vine_analysis_df.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for NON-VINE CUSTOMERS")
non_vine_summary_analysis_df.show()

Summary statistics for NON-VINE CUSTOMERS
+-------+------------------+------------------+------------------+
|summary|       star_rating|     helpful_votes|       total_votes|
+-------+------------------+------------------+------------------+
|  count|           2299036|           2299036|           2299036|
|   mean| 4.239844004182623|1.8453164717733868| 2.292603943565912|
| stddev|1.2106067274034567|12.770884193978548|13.628641248520951|
|    min|                 1|                 0|                 0|
|    max|                 5|              2703|              2751|
+-------+------------------+------------------+------------------+



In [ ]:
# Top 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS

from pyspark.sql.functions import desc

non_vine_helpful_votes_df = non_vine_analysis_df.orderBy(non_vine_analysis_df.helpful_votes.desc())
non_vine_helpful_votes_df = non_vine_helpful_votes_df.filter('star_rating = 5')

print("Top 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS")
non_vine_helpful_votes_df.show(20)


Top 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2011-01-13 00:10:00|   16316106|                Y|Magnet Steel Bike...|          5|         2478|       2600|   N|
|2014-01-19 00:04:00|   34490348|                Y|LifeStraw Persona...|          5|         2362|       2496|   N|
|2013-01-04 00:10:00|   30530118|                Y|SoundAsleep Dream...|          5|         2350|       2420|   N|
|2013-01-18 00:05:00|   35554264|                Y|LifeStraw Persona...|          5|         1864|       1961|   N|
|2013-01-24 00:10:00|    6262999|                Y|SoundAsleep Dream...|          5|         1722|       1792|   N|


In [ ]:
#  Converting into Pandas DataFrame to get the complete product title name 
non_vine_helpful_votes_df.select("product_title").toPandas().head(3)
# Confirmed full name on Amazon

,product_title
0,Magnet Steel Bike Bicycle Indoor Exercise Trai...
1,"LifeStraw Personal Water Filter for Hiking, Ca..."
2,SoundAsleep Dream Series Air Mattress with Com...


In [ ]:
# Worst 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS

from pyspark.sql.functions import desc

low_non_vine_helpful_votes_df = non_vine_analysis_df.orderBy(non_vine_analysis_df.helpful_votes.desc())
low_non_vine_helpful_votes_df = low_non_vine_helpful_votes_df.filter('star_rating = 1')

print("Worst 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS")
low_non_vine_helpful_votes_df.show()


Worst 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2014-01-17 00:05:00|   42216608|                N|Sawyer Products B...|          1|         2663|       2701|   N|
|2011-01-22 00:11:00|   52259140|                N|Defence Technolog...|          1|         2449|       2556|   N|
|2014-01-06 00:11:00|   18195395|                Y|Kryptonite Krypto...|          1|         1682|       1735|   N|
|2013-01-03 00:07:00|   47476756|                Y|Kryptonite Krypto...|          1|         1588|       1684|   N|
|2011-01-23 00:11:00|   18857489|                N|Defence Technolog...|          1|         1266|       1330|   N

In [ ]:
#  Converting into Pandas DataFrame to get the complete product title name 
low_non_vine_helpful_votes_df.select("product_title").toPandas().head(3)
# Confirmed full name on Amazon

,product_title
0,Sawyer Products B4 Extractor Pump Kit
1,Defence Technology First Defense 1.3% MK-9 Str...
2,Kryptonite KryptoLok Series 2 Standard Heavy D...


### Comparing Top 3 helpful voted products with a five star rating from VINE CUSTOMERS with rating from NON-VINE CUSTOMERS

In [ ]:
#  5 star rated product with the number 1 top helpful votes(326): Thule Urban Glide - Jogging Stroller
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

# Source Link: http://www.datasciencemadesimple.com/subset-or-filter-data-with-multiple-conditions-in-pyspark/

import pyspark.sql.functions as f

top_1_product_helpful_df = non_vine_helpful_votes_df.filter((f.col('product_title')=='Thule Urban Glide - Jogging Stroller'))
thule_summary_analysis_df = top_1_product_helpful_df.select(["star_rating"]).describe()

thule_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|         10|
|   mean|        5.0|
| stddev|        0.0|
|    min|          5|
|    max|          5|
+-------+-----------+



In [ ]:
#  5 star rated product with the number 2 top helpful votes(170): Zippo Flex Neck Utility Lighter
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

top_2_product_helpful_df = non_vine_helpful_votes_df.filter((f.col('product_title')=='Zippo Flex Neck Utility Lighter'))
zippo_summary_analysis_df = top_2_product_helpful_df.select(["star_rating"]).describe()

zippo_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|        133|
|   mean|        5.0|
| stddev|        0.0|
|    min|          5|
|    max|          5|
+-------+-----------+



In [ ]:
#  5 star rated product with the number 3 top helpful votes(140): YBIKE Balance Bike
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

top_3_product_helpful_df = non_vine_helpful_votes_df.filter((f.col('product_title')=='YBIKE Balance Bike'))
ybike_summary_analysis_df = top_3_product_helpful_df.select(["star_rating"]).describe()

ybike_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|         99|
|   mean|        5.0|
| stddev|        0.0|
|    min|          5|
|    max|          5|
+-------+-----------+



### Comparing Top 3 helpful voted products with a one star rating from VINE CUSTOMERS with rating from NON-VINE CUSTOMERS

In [ ]:
#  1 star rated product with the number 1 top helpful votes(18): Runtastic Bike Case for iPhone 4, 4S, 5 (Black)
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

low_1_product_helpful_df = low_non_vine_helpful_votes_df.filter((f.col('product_title')=='Runtastic Bike Case for iPhone 4, 4S, 5 (Black)'))
case_summary_analysis_df = low_1_product_helpful_df.select(["star_rating"]).describe()

case_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



In [ ]:
#  1 star rated product with the number 2 top helpful votes(17): Allen Sports Premier 4-Bike Trunk Rack
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

low_2_product_helpful_df = low_non_vine_helpful_votes_df.filter((f.col('product_title')=='Allen Sports Premier 4-Bike Trunk Rack'))
rack_summary_analysis_df = low_2_product_helpful_df.select(["star_rating"]).describe()

rack_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          6|
|   mean|        1.0|
| stddev|        0.0|
|    min|          1|
|    max|          1|
+-------+-----------+



In [ ]:
#  1 star rated product with the number 3 top helpful votes(11): Wahoo Balance Bluetooth Smart Scale for iPhone and iPad 
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

low_3_product_helpful_df = low_non_vine_helpful_votes_df.filter((f.col('product_title')=='Wahoo Balance Bluetooth Smart Scale for iPhone and iPad '))
scale_summary_analysis_df = low_3_product_helpful_df.select(["star_rating"]).describe()

scale_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



### Comparing Top 3 helpful voted products with a five star rating from NON-VINE CUSTOMERS with rating from VINE CUSTOMERS

In [ ]:
#  5 star rated product with the number 1 top helpful votes(2478): Magnet Steel Bike Bicycle Indoor Exercise Trainer
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

vine_top_1_product_helpful_df = vine_helpful_votes_df.filter((f.col('product_title')=='Magnet Steel Bike Bicycle Indoor Exercise Trainer'))
magbike_summary_analysis_df = vine_top_1_product_helpful_df.select(["star_rating"]).describe()

magbike_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



In [ ]:
#  5 star rated product with the number 2 top helpful votes(2362): LifeStraw Personal Water Filter for Hiking, Camping, Travel, and Emergency Preparedness
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

vine_top_2_product_helpful_df = vine_helpful_votes_df.filter((f.col('product_title')=='LifeStraw Personal Water Filter for Hiking, Camping, Travel, and Emergency Preparedness'))
lifestraw_summary_analysis_df = vine_top_2_product_helpful_df.select(["star_rating"]).describe()

lifestraw_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



In [ ]:
#  5 star rated product with the number 3 top helpful votes(2350): SoundAsleep Dream Series Air Mattress with ComfortCoil Technology & Internal High Capacity Pump - Queen Size
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

vine_top_3_product_helpful_df = vine_helpful_votes_df.filter((f.col('product_title')=='SoundAsleep Dream Series Air Mattress with ComfortCoil Technology & Internal High Capacity Pump - Queen Size'))
matress_summary_analysis_df = vine_top_3_product_helpful_df.select(["star_rating"]).describe()

matress_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



### Comparing Top 3 helpful voted products with a one star rating from NON-VINE CUSTOMERS with rating from VINE CUSTOMERS

In [ ]:
#  1 star rated product with the number 1 top helpful votes(2663): Sawyer Products B4 Extractor Pump Kit
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

vine_low_1_product_helpful_df = low_vine_helpful_votes_df.filter((f.col('product_title')=='Sawyer Products B4 Extractor Pump Kit'))
pump_summary_analysis_df = vine_low_1_product_helpful_df.select(["star_rating"]).describe()

pump_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



In [ ]:
#  1 star rated product with the number 2 top helpful votes(2449): Defence Technology First Defense 1.3% MK-9 Stream OC Aerosol
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

low_2_product_helpful_df = low_vine_helpful_votes_df.filter((f.col('product_title')=='Defence Technology First Defense 1.3% MK-9 Stream OC Aerosol'))
aerosol_summary_analysis_df = low_2_product_helpful_df.select(["star_rating"]).describe()

aerosol_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



In [ ]:
#  1 star rated product with the number 3 top helpful votes(1682): Wahoo Balance Bluetooth Smart Scale for iPhone and iPad 
#  Looked into what the summary statistics were for the same product rated by NON-VINE customers

low_3_product_helpful_df = low_vine_helpful_votes_df.filter((f.col('product_title')=='Kryptonite KryptoLok Series 2 Standard Heavy Duty Bicycle U Lock with 4ft Flex Bike Cable'))
bikecable_summary_analysis_df = low_3_product_helpful_df.select(["star_rating"]).describe()

bikecable_summary_analysis_df.show()

+-------+-----------+
|summary|star_rating|
+-------+-----------+
|  count|          0|
|   mean|       null|
| stddev|       null|
|    min|       null|
|    max|       null|
+-------+-----------+



## Count of Vine Customers and Non-Vine Customers

In [ ]:
#  Count of VINE CUSTOMERS

vine_customers_df = vine_analysis_df.groupby("vine").agg({"vine":"count"})
vine_customers_df.show()

+----+-----------+
|vine|count(vine)|
+----+-----------+
|   Y|       3137|
+----+-----------+



In [ ]:
#  Count of NON-VINE CUSTOMERS

non_vine_customers_df = non_vine_analysis_df.groupby("vine").agg({"vine":"count"})
non_vine_customers_df.show()

+----+-----------+
|vine|count(vine)|
+----+-----------+
|   N|    2299036|
+----+-----------+



In [ ]:
# Combine customer count 
# Source link: https://sparkbyexamples.com/spark/spark-dataframe-union-and-union-all/

total_customers_df = vine_customers_df.unionAll(non_vine_customers_df)
total_customers_df = total_customers_df.withColumnRenamed('count(vine)', 'Number of customers')

print("Number of Vine and Non-Vine customers")
total_customers_df.show()

Number of Vine and Non-Vine customers
+----+-------------------+
|vine|Number of customers|
+----+-------------------+
|   Y|               3137|
|   N|            2299036|
+----+-------------------+



## Number of Reviews made by the top rated and most helpful VINE CUSTOMERS 

In [ ]:
#  Customer: 51999404

view_count_topvine_customer_df = vine_helpful_votes_df.filter((f.col('customer_id')=="51999404"))
view_count_topvine_customer_df.show()

+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2014-01-08 00:08:00|   51999404|                N|Thule Urban Glide...|          5|          326|        340|   Y|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+



In [ ]:
#  Customer: 49805190

view_count_2topvine_customer_df = vine_helpful_votes_df.filter((f.col('customer_id')=="49805190"))
view_count_2topvine_customer_df.show()

+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2012-01-01 00:08:00|   49805190|                N|Zippo Flex Neck U...|          5|          170|        181|   Y|
|2010-01-22 00:04:00|   49805190|                N|Optic Nerve Eyequ...|          5|            2|          2|   Y|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+



In [ ]:
#  Customer: 51859664

view_count_3topvine_customer_df = vine_helpful_votes_df.filter((f.col('customer_id')=="51859664"))
view_count_3topvine_customer_df.show()

+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2010-01-19 00:09:00|   51859664|                N|  YBIKE Balance Bike|          5|          140|        153|   Y|
|2014-01-03 00:02:00|   51859664|                N|Thule  Cougar One...|          5|            0|          0|   Y|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+



# Type of Reviews by customers with highest review count



In [ ]:
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13355404|           291|
|   47355039|           205|
|   51155788|           197|
|   20433453|           129|
|   40944345|           128|
|    3247563|           128|
|   27550730|           125|
|   30636778|           115|
|   10796373|           113|
|   40364490|           112|
|   16255502|           112|
|   38137849|           111|
|   52988261|           109|
|   36205928|           108|
|   18466687|           104|
|   52196389|           104|
|   44142946|           102|
|   49782074|           100|
|   45014035|           100|
|   31807575|            98|
+-----------+--------------+
only showing top 20 rows



In [ ]:
#  Customer: 13355404

customer_review_count = non_vine_helpful_votes_df.filter((f.col('customer_id')=="13355404"))
customer_review_count.show(3)

+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2014-01-18 00:04:00|   13355404|                N|Bushbox Outdoor P...|          5|          183|        189|   N|
|2015-01-19 00:06:00|   13355404|                N|OutdoorMaster Hik...|          5|          129|        137|   N|
|2014-01-14 00:04:00|   13355404|                N|Bushbox Titanium ...|          5|           84|         86|   N|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
only showing top 3 rows



In [ ]:
#  Customer: 47355039

customer2_review_count = non_vine_helpful_votes_df.filter((f.col('customer_id')=="47355039"))
customer2_review_count.show(3)

+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2009-01-05 00:11:00|   47355039|                N|Adult Trike Seat ...|          5|           12|         13|   N|
|2009-01-26 00:08:00|   47355039|                Y|Acera BR-M422 fro...|          5|            7|          7|   N|
|2009-01-13 00:07:00|   47355039|                N|Bell iPulse LED B...|          5|            7|          9|   N|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
only showing top 3 rows



In [ ]:
#  Customer: 51155788

customer2_review_count = non_vine_helpful_votes_df.filter((f.col('customer_id')=="51155788"))
customer2_review_count.show(3)

+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|        review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|2014-01-12 00:02:00|   51155788|                Y|SRAM Supercork Bi...|          5|           32|         34|   N|
|2014-01-03 00:03:00|   51155788|                Y|Selle Royal Respi...|          5|           25|         27|   N|
|2013-01-19 00:06:00|   51155788|                Y|Kenda Tube Bicycl...|          5|           19|         22|   N|
+-------------------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
only showing top 3 rows

